In [ ]:
# Install required libraries
!pip install python-dotenv PyPDF2 requests

# Import necessary libraries
import os
import PyPDF2
import requests
from google.colab import files
from dotenv import load_dotenv

# Step 1: Create and load the .env file for the API key
# Replace the placeholder with your actual API key
with open(".env", "w") as f:
    f.write("GEMINI_API_KEY=your_actual_gemini_api_key_here")

load_dotenv()  # Load variables from .env file
gemini_api_key = os.getenv("GEMINI_API_KEY")

# Validate API key loading
if not gemini_api_key:
    raise ValueError("GEMINI_API_KEY not found in environment variables.")

# Step 2: Upload the PDF file
uploaded = files.upload()

# Step 3: Extract text from the uploaded PDF
def extract_text_from_pdf(pdf_file):
    with open(pdf_file, "rb") as file:
        reader = PyPDF2.PdfReader(file)
        text = ""
        for page in reader.pages:
            text += page.extract_text()
    return text

# Get the filename and extract text
pdf_filename = list(uploaded.keys())[0]
pdf_text = extract_text_from_pdf(pdf_filename)

# Step 4: Set up Gemini API endpoint
gemini_endpoint = 'https://generativelanguage.googleapis.com/v1beta/models/gemini-2.0-flash:generateContent'

# Step 5: Define function to interact with Gemini API
def chat_with_pdf(pdf_text, query):
    data = {
        "contents": [{
            "parts": [{"text": f"Given the following document, answer the question:\n\n{pdf_text}\n\nQuestion: {query}"}]
        }]
    }

    headers = {
        'Content-Type': 'application/json',
    }

    response = requests.post(f"{gemini_endpoint}?key={gemini_api_key}", json=data, headers=headers)

    if response.status_code == 200:
        try:
            return response.json()['candidates'][0]['content']['parts'][0]['text'].strip()
        except KeyError as e:
            return f"Error: Key {str(e)} not found in the response. Check response structure:\n{response.json()}"
    else:
        return f"Error: {response.status_code}, {response.text}"

# Step 6: Interactive chat interface
while True:
    user_query = input("Ask a question or type 'exit' to quit: ")
    if user_query.lower() == 'exit':
        break
    response = chat_with_pdf(pdf_text, user_query)
    print(f"Chatbot: {response}")
